In [1]:
import gym
import numpy
import scipy
env = gym.make('Taxi-v2')
new_state = env.reset()
from copy import copy

[2017-11-30 12:13:11,628] Making new env: Taxi-v2


In [2]:
observation = env.reset()
for i in range(100):
    #env.render()
    #print (observation)
    action = env.action_space.sample()
    observation, reward, done, info = env.step(action)
    if done:
        print ("Done")
        break

In [70]:
q_values = numpy.zeros((env.observation_space.n, env.action_space.n))
s_values = numpy.zeros((env.observation_space.n))

In [4]:
def q_learn(iterations, q_values):
    total_reward = 0
    for i in range(iterations):
        new_state = env.reset()
        state = new_state 
        while True:
            action = numpy.argmax(q_values[state])
            new_state, reward, done, info = env.step(action)
            q_values[state][action] = reward + 0.5*numpy.max(q_values[new_state]) + 0.5*q_values[state][action]
            state = new_state
            total_reward += reward
            if done:
                break
    return q_values, total_reward

In [112]:
def q_action(observation, env):
    value_list = numpy.zeros(env.action_space.n)
    for i in range(env.action_space.n):
        state = copy(env)
        new_observation, reward, done, info = state.step(i)
        value_list[i] = s_values[new_observation] + reward
    action = numpy.random.choice(numpy.flatnonzero(value_list == value_list.max()))
    q_values[observation][action] = (1 - alpha)*q_values
    return action

In [ ]:
def dls(pos, curr_depth, max_depth, mb_internal_environment):
    value_list = list()
    if curr_depth == max_depth:
        return mb_internal_environment_value(pos, mb_internal_environment)
    else:
        actions = get_legal_actions(pos)
        for i in actions:
            next_pos = numpy.add(numpy.array(pos), numpy.array(i))
            value = mb_internal_environment_value(pos, mb_internal_environment) + gamma * dls(next_pos, curr_depth+1, max_depth, mb_internal_environment)
            value_list.append(value)
        if value_list:
            average_value = numpy.mean(numpy.array(value_list))
    return average_value

def dls_action(pos, max_depth, mb_internal_environment):
    # Do depth Limited Search till max_depth on the mb_internal_environment
    actions = get_legal_actions(pos)
    next_value_list = list()
    next_pos_list = list()
    for i in actions:
        next_pos = numpy.add(numpy.array(pos), numpy.array(i))
        next_value_list.append(dls(next_pos, 0, max_depth, mb_internal_environment))
        next_pos_list.append(next_pos)
    zipped = list(zip(actions, next_value_list))
    random.shuffle(zipped)
    actions, next_value_list = zip(*zipped)
    action = actions[action_from_softmax(softmax(next_value_list))]
    return action

[2017-11-30 15:20:03,366] Making new env: Taxi-v2


rec_dir: rec/11
Taxi-v2
  1   score:    -1.000   avg_time: 0.0 s

In [11]:
print (env.action_space.n)

6


In [ ]:
print (q_learn(360, q_values))

In [ ]:
env_state = copy(env)

In [ ]:
state_values = numpy.max(q_values, axis=1)
print (len(state_values))

In [65]:
state = env.observation_space.sample()
action = env.action_space.sample()
state = env.reset()
print (state)
print (action)
observation, reward, done, something = env.step(action)
print (observation)

13
0
113


In [113]:
print (q_action(observation, env))

[ -1.  -1.  -1.  -1. -10. -10.]
0
